# 02a - Cylinder Segmentation Demo

This notebook demonstrates mesh segmentation using a simple cylinder geometry. The cylinder represents the simplest case with genus = 0 (no holes).

**Part of the GenCoMo Demo Series** - [Return to Index](01_tutorial_index.ipynb)

In [1]:
import numpy as np
import trimesh
from gencomo.demos import create_cylinder_mesh
from gencomo.segmentation import MeshSegmenter
from gencomo import visualize_mesh_3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Create and Visualize Cylinder

In [2]:
# Create a cylinder mesh
cylinder = create_cylinder_mesh(radius=1, length=2, resolution=32)

print(f"Cylinder properties:")
print(f"  Volume: {cylinder.volume:.3f}")
print(f"  Surface area: {cylinder.area:.3f}")
print(f"  Z-bounds: {cylinder.bounds[:, 2]}")

# Visualize the original cylinder
fig = visualize_mesh_3d(cylinder, title="Original Cylinder", backend="plotly")
fig.show()

Cylinder properties:
  Volume: 6.243
  Surface area: 18.789
  Z-bounds: [-1.  1.]


## Segment the Mesh

In [3]:
# Create segmenter and segment the mesh
segmenter = MeshSegmenter()
segments = segmenter.segment_mesh(cylinder, slice_width=0.5, min_volume=0.1)

print(f"Segmentation complete!")
print(f"Total segments: {len(segments)}")
print(f"Total slices: {len(segmenter.slices)}")

Segmenting mesh (z: -1.000 to 1.000) into 4 slices
Created 4 segments across 4 slices
Segmentation complete!
Total segments: 4
Total slices: 4


## Analyze Segments

In [4]:
# Show segments per slice
print("Segments per slice:")
for i in range(len(segmenter.slices)):
    slice_segments = segmenter.get_segments_in_slice(i)
    print(f"  Slice {i}: {len(slice_segments)} segments")

stats = segmenter.compute_segmentation_statistics()

print(f"\nTotal volume: {stats['volume_stats']['total']:.4f}")
print(f"Mean segment volume: {stats['volume_stats']['mean']:.4f}")
print(f"Connected components: {stats['connected_components']}")

Segments per slice:
  Slice 0: 1 segments
  Slice 1: 1 segments
  Slice 2: 1 segments
  Slice 3: 1 segments

Total volume: 6.2429
Mean segment volume: 1.5607
Connected components: 1


In [5]:
for seg in segments:
    print(f"Segment ID: {seg.id}")
    print(f"  Slice Index: {seg.slice_index}")
    print(f"  Segment Index: {seg.segment_index}")
    print(f"  Volume: {seg.volume:.4f}")
    print(f"  Exterior Area: {seg.exterior_surface_area:.4f}")
    print(f"  Interior Area: {seg.interior_surface_area:.4f}")
    print(f"  Center:", [f"{coord:.4f}" for coord in seg.centroid])
    print(f"  Z Range: [{seg.z_min:.4f}, {seg.z_max:.4f}]")
    print(f"  Connected Component: {seg.is_connected_component}")
    print("-" * 40)

Segment ID: slice_0_seg_0
  Slice Index: 0
  Segment Index: 0
  Volume: 1.5607
  Exterior Area: 6.2580
  Interior Area: 3.1214
  Center: ['-0.0000', '-0.0000', '-0.7500']
  Z Range: [-1.0000, -0.5000]
  Connected Component: True
----------------------------------------
Segment ID: slice_1_seg_0
  Slice Index: 1
  Segment Index: 0
  Volume: 1.5607
  Exterior Area: 3.1365
  Interior Area: 6.2429
  Center: ['-0.0000', '-0.0000', '-0.2500']
  Z Range: [-0.5000, 0.0000]
  Connected Component: True
----------------------------------------
Segment ID: slice_2_seg_0
  Slice Index: 2
  Segment Index: 0
  Volume: 1.5607
  Exterior Area: 3.1365
  Interior Area: 6.2429
  Center: ['-0.0000', '-0.0000', '0.2500']
  Z Range: [0.0000, 0.5000]
  Connected Component: True
----------------------------------------
Segment ID: slice_3_seg_0
  Slice Index: 3
  Segment Index: 0
  Volume: 1.5607
  Exterior Area: 6.2580
  Interior Area: 3.1214
  Center: ['-0.0000', '-0.0000', '0.7500']
  Z Range: [0.5000, 1.00

## Visualize Connectivity

In [6]:
# Visualize connectivity in 3D space
try:
    segmenter.visualize_connectivity_graph_3d(backend="plotly")
except Exception as e:
    print(f"3D visualization not available: {e}")
    print("You can install plotly for 3D visualization: pip install plotly")

## Summary

The cylinder segmentation demonstrates:

- **Simple topology**: Each slice contains one segment (genus = 0)
- **Linear connectivity**: Segments connect sequentially along the z-axis  
- **Accurate volume conservation**: Total volume is preserved during segmentation
- **Correct surface area classification**: Interior (cut) vs exterior (original) surfaces
- **Predictable geometry**: Results match theoretical calculations

This represents the baseline case for mesh segmentation and validates the core algorithm functionality.